In [1]:
import math
import requests
import datetime
from statistics import mean
import time
import sys

YEAR = 2020
WEEK = 14
POST_WEEK = 0
LI_RATIO = .1
L_SHARE = LI_RATIO
I_SHARE = 1 - LI_RATIO
TRANSFER_RATIO = 1
MARGIN_LOG = 7

In [2]:
#prev = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=regular')
#prev_post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=postseason')
reg = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=regular')
post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=postseason')
records = {}
games = reg.json() + post.json() #+ prev.json() + prev_post.json()

In [3]:
for game in games:
        
    away_team = game['away_team']
    home_team = game['home_team']
    
    # With this dataset, no conference listed means they are an FCS team. 
    # I treat all FCS teams as the same, likely crappy team. 
    if not game['away_conference']:
        away_team = 'FCS Team'
    if not game['home_conference']:
        home_team = 'FCS Team'   
    
    if away_team not in records:
        records[away_team] = []
    if home_team not in records:
        records[home_team] = []
    
    if ((game['season_type'] != 'postseason'and game['week'] > WEEK) 
        or (game['season_type'] == 'postseason' and game['week'] > POST_WEEK)):
        continue
    
    # For invalid/unplayed games
    if game['away_points'] == None:
        continue
        
    margin = game['away_points'] - game['home_points']
    
    # Going to overtime counts as a one point win
    if len(game['away_line_scores']) > 4:
        margin = margin / abs(margin)
    records[away_team].append((home_team, margin))
    records[home_team].append((away_team, -margin))

In [9]:
AP_Ranks = {}
for team in records:
    AP_Ranks[team] = '-'
reg = requests.get('https://api.collegefootballdata.com/rankings?year=' + str(YEAR) 
                   + '&week=' + str(WEEK + POST_WEEK + 1)
                   + '&seasonType=regular')
for e in reg.json()[0]['polls']:
    if e['poll'] == 'AP Top 25':
        for row in e['ranks']:
            AP_Ranks[row['school']] = str(row['rank'])

In [5]:
records['Ohio State']

[('Nebraska', 35),
 ('Penn State', 13),
 ('Rutgers', 22),
 ('Indiana', 7),
 ('Michigan State', 40)]

In [6]:
# Get last weeks result!
last_week_table = {}
with open('Output - ' + str(YEAR) + '-' + str(WEEK + POST_WEEK - 1) + '.md', 'r') as last_week:
    rows = last_week.read().split('\n')
    header = []
    for row in rows:
        if '|' in row:
            cells = row.split('|')
            if header == []:
                header = cells
                team_loc = header.index('Team')
            else:
                team = '('.join(cells[team_loc].split('(')[0:-1]) # Get the team name without record, but with (OH)
                last_week_table[team] = {}
                for i, cell in enumerate(cells):
                    if cell == '':
                        continue
                    last_week_table[team][header[i]] = cell
                
                
                

In [7]:
last_week_table

{'': {'Rank': '-',
  'Score': '-',
  'Team': '-',
  'AP Rank': '-',
  'Past Rank': '-',
  'Score Change': '-',
  'Breakdown': '-'},
 'BYU': {'Rank': '1',
  'Score': '1384',
  'Team': 'BYU(9-0)',
  'AP Rank': '8',
  'Past Rank': '1',
  'Score Change': '-63',
  'Breakdown': 'See Link'},
 'Cincinnati': {'Rank': '2',
  'Score': '1322',
  'Team': 'Cincinnati(8-0)',
  'AP Rank': '7',
  'Past Rank': '2',
  'Score Change': '-31',
  'Breakdown': 'See Link'},
 'Alabama': {'Rank': '3',
  'Score': '1277',
  'Team': 'Alabama(7-0)',
  'AP Rank': '1',
  'Past Rank': '4',
  'Score Change': '+205',
  'Breakdown': 'See Link'},
 'Coastal Carolina': {'Rank': '4',
  'Score': '1092',
  'Team': 'Coastal Carolina(8-0)',
  'AP Rank': '16',
  'Past Rank': '3',
  'Score Change': '-158',
  'Breakdown': 'See Link'},
 'Clemson': {'Rank': '5',
  'Score': '1068',
  'Team': 'Clemson(7-1)',
  'AP Rank': '4',
  'Past Rank': '6',
  'Score Change': '+96',
  'Breakdown': 'See Link'},
 'Notre Dame': {'Rank': '6',
  'Score':

In [7]:
start = time.time()
batch_start = time.time()

rankings = {}
scores = {}
for team in records:
    rankings[team] = 0
    scores[team] = [0]

team_count = len(rankings.keys())

rank_history = [] # used for checking if we have hit a repeating pattern based on ranks
past_averaged_history = '' # used to see if the patterns are broadly repeating
rank_history_set = set() 
score_history = [] # used for easier lookups of what team scores were in said repeating pattern
averaged_score_history = [] 
scores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)
repeat_point = 0
pattern_found = False
# Repeat for 100000 interations, but it should hit a repeating steady state before then
for i in range(1000001):
    for team in records.keys():
        scores[team] = [0] # the first value is the total score
        games = records[team]
        for game in games:
            opponent = game[0]
            margin = game[1]
            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1
            multiplier *= 1000 # just for nicer numbers
            game_score = 0
            if margin < 0:
                game_score -= L_SHARE * rankings[opponent] / team_count
                game_score -= I_SHARE / (team_count - rankings[opponent])
                if rankings[opponent] > rankings[team]:
                    game_score -= TRANSFER_RATIO * L_SHARE * (team_count - rankings[team]) / team_count
                    game_score -= TRANSFER_RATIO * I_SHARE / (1 + rankings[team])
            if margin > 0:
                game_score += L_SHARE * (team_count - rankings[opponent]) / team_count
                game_score += I_SHARE / (1 + rankings[opponent])
                if rankings[opponent] < rankings[team]:
                    game_score += TRANSFER_RATIO * L_SHARE * rankings[team] / team_count
                    game_score += TRANSFER_RATIO * I_SHARE / (team_count - rankings[team])
            scores[team] += [multiplier * game_score]
        if i == 0:
            scores[team][0] = sum(scores[team])
        elif i == 1:
            scores[team][0] = (sum(scores[team]) + score_history[-1][team][0]) / 2
        else:
            scores[team][0] = (sum(scores[team]) 
                               + (2 * score_history[-1][team][0] 
                                  - score_history[-2][team][0])) / 2
    # This is sorted, but adjusting so tied teams have the same (higher) rank
    new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
    #print(new_rank_orders)
    new_ranks = rankings
    new_ranks[new_rank_orders[0]] = 0
    past_rank = 0
    past_score = scores[new_rank_orders[0]][0]
    for rank, team in enumerate(new_rank_orders):
        if (abs(scores[team][0] - past_score) < 0.01):
            new_ranks[team] = past_rank
        else:
            new_ranks[team] = rank
        past_rank = new_ranks[team]
        past_score = scores[team][0]
    # stop if we are repeating ourselves, record this if we are not
    if False:
        repeat_point = rank_history.index(str(new_ranks))
        pattern_found = True
        break
    else:
        rank_history += [str(new_ranks)]
        rank_history_set.add(str(new_ranks))
        score_history += [scores.copy()]
        rankings = new_ranks
        
    # stop if we are *broadly* repeating ourselves
    if i%1000 == 0 and i != 0:
        print(i, 'Iterations Took: ' + str(round(time.time() - batch_start)))
        check_start = time.time()
        
        for team in records:
            score = [x[team] for x in score_history]
            scores[team] = list(map(mean, zip(*score)))
            scores[team][0] = sum(scores[team][1:]) 
        new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
        if str(new_rank_orders) == past_averaged_history:
            pattern_found = True
            break
        else:
            averaged_score_history += [scores.copy()]
            past_averaged_history = str(new_rank_orders)
        print(i, 'Check Took: ' + str(round(time.time() - check_start)))
        batch_start = time.time()
        
        
        
    if i>=1000:
        old_news = rank_history[0]
        del rank_history[0]
        del score_history[0]
            

print(i)



1000 Iterations Took: 8
1000 Check Took: 4
2000 Iterations Took: 8
2000 Check Took: 6
3000 Iterations Took: 7
3000 Check Took: 4
4000 Iterations Took: 6
4000 Check Took: 3
5000 Iterations Took: 5
5000 Check Took: 3
6000 Iterations Took: 5
6000 Check Took: 3
7000 Iterations Took: 5
7000 Check Took: 4
8000 Iterations Took: 5
8000 Check Took: 3
9000 Iterations Took: 6
9000 Check Took: 3
10000 Iterations Took: 5
10000 Check Took: 4
11000 Iterations Took: 5
11000 Check Took: 3
12000 Iterations Took: 5
12000 Check Took: 3
13000 Iterations Took: 5
13000 Check Took: 3
14000 Iterations Took: 5
14000 Check Took: 3
15000 Iterations Took: 5
15000 Check Took: 4
16000 Iterations Took: 7
16000 Check Took: 4
17000 Iterations Took: 7
17000 Check Took: 4
18000 Iterations Took: 8
18000 Check Took: 5
19000 Iterations Took: 7
19000 Check Took: 4
20000 Iterations Took: 6
20000 Check Took: 4
21000 Iterations Took: 7
21000 Check Took: 4
22000 Iterations Took: 8
22000 Check Took: 5
23000 Iterations Took: 7
230

180000 Iterations Took: 5
180000 Check Took: 3
181000 Iterations Took: 5
181000 Check Took: 3
182000 Iterations Took: 5
182000 Check Took: 3
183000 Iterations Took: 5
183000 Check Took: 3
184000 Iterations Took: 5
184000 Check Took: 3
185000 Iterations Took: 5
185000 Check Took: 3
186000 Iterations Took: 5
186000 Check Took: 3
187000 Iterations Took: 5
187000 Check Took: 3
188000 Iterations Took: 5
188000 Check Took: 3
189000 Iterations Took: 5
189000 Check Took: 3
190000 Iterations Took: 5
190000 Check Took: 3
191000 Iterations Took: 5
191000 Check Took: 3
192000 Iterations Took: 6
192000 Check Took: 3
193000 Iterations Took: 6
193000 Check Took: 4
194000 Iterations Took: 6
194000 Check Took: 4
195000 Iterations Took: 5
195000 Check Took: 3
196000 Iterations Took: 5
196000 Check Took: 3
197000 Iterations Took: 5
197000 Check Took: 4
198000 Iterations Took: 6
198000 Check Took: 3
199000 Iterations Took: 6
199000 Check Took: 3
200000 Iterations Took: 5
200000 Check Took: 3
201000 Iterat

354000 Check Took: 2
355000 Iterations Took: 2
355000 Check Took: 1
356000 Iterations Took: 2
356000 Check Took: 2
357000 Iterations Took: 2
357000 Check Took: 2
358000 Iterations Took: 2
358000 Check Took: 2
359000 Iterations Took: 2
359000 Check Took: 2
360000 Iterations Took: 2
360000 Check Took: 2
361000 Iterations Took: 2
361000 Check Took: 2
362000 Iterations Took: 2
362000 Check Took: 2
363000 Iterations Took: 2
363000 Check Took: 2
364000 Iterations Took: 2
364000 Check Took: 2
365000 Iterations Took: 2
365000 Check Took: 2
366000 Iterations Took: 2
366000 Check Took: 2
367000 Iterations Took: 2
367000 Check Took: 2
368000 Iterations Took: 2
368000 Check Took: 2
369000 Iterations Took: 2
369000 Check Took: 2
370000 Iterations Took: 2
370000 Check Took: 2
371000 Iterations Took: 2
371000 Check Took: 2
372000 Iterations Took: 2
372000 Check Took: 2
373000 Iterations Took: 2
373000 Check Took: 2
374000 Iterations Took: 2
374000 Check Took: 2
375000 Iterations Took: 2
375000 Check 

529000 Iterations Took: 2
529000 Check Took: 2
530000 Iterations Took: 2
530000 Check Took: 2
531000 Iterations Took: 3
531000 Check Took: 2
532000 Iterations Took: 3
532000 Check Took: 2
533000 Iterations Took: 3
533000 Check Took: 2
534000 Iterations Took: 2
534000 Check Took: 2
535000 Iterations Took: 3
535000 Check Took: 2
536000 Iterations Took: 3
536000 Check Took: 2
537000 Iterations Took: 3
537000 Check Took: 2
538000 Iterations Took: 2
538000 Check Took: 2
539000 Iterations Took: 3
539000 Check Took: 2
540000 Iterations Took: 3
540000 Check Took: 2
541000 Iterations Took: 3
541000 Check Took: 2
542000 Iterations Took: 3
542000 Check Took: 2
543000 Iterations Took: 3
543000 Check Took: 2
544000 Iterations Took: 3
544000 Check Took: 2
545000 Iterations Took: 3
545000 Check Took: 2
546000 Iterations Took: 3
546000 Check Took: 2
547000 Iterations Took: 3
547000 Check Took: 2
548000 Iterations Took: 3
548000 Check Took: 2
549000 Iterations Took: 3
549000 Check Took: 2
550000 Iterat

703000 Check Took: 2
704000 Iterations Took: 2
704000 Check Took: 2
705000 Iterations Took: 2
705000 Check Took: 2
706000 Iterations Took: 2
706000 Check Took: 2
707000 Iterations Took: 2
707000 Check Took: 2
708000 Iterations Took: 2
708000 Check Took: 2
709000 Iterations Took: 2
709000 Check Took: 2
710000 Iterations Took: 2
710000 Check Took: 2
711000 Iterations Took: 2
711000 Check Took: 2
712000 Iterations Took: 3
712000 Check Took: 2
713000 Iterations Took: 2
713000 Check Took: 2
714000 Iterations Took: 2
714000 Check Took: 2
715000 Iterations Took: 2
715000 Check Took: 2
716000 Iterations Took: 2
716000 Check Took: 2
717000 Iterations Took: 2
717000 Check Took: 2
718000 Iterations Took: 2
718000 Check Took: 2
719000 Iterations Took: 2
719000 Check Took: 2
720000 Iterations Took: 3
720000 Check Took: 2
721000 Iterations Took: 2
721000 Check Took: 2
722000 Iterations Took: 2
722000 Check Took: 2
723000 Iterations Took: 2
723000 Check Took: 2
724000 Iterations Took: 2
724000 Check 

878000 Iterations Took: 3
878000 Check Took: 2
879000 Iterations Took: 2
879000 Check Took: 2
880000 Iterations Took: 2
880000 Check Took: 2
881000 Iterations Took: 2
881000 Check Took: 2
882000 Iterations Took: 3
882000 Check Took: 2
883000 Iterations Took: 2
883000 Check Took: 2
884000 Iterations Took: 3
884000 Check Took: 2
885000 Iterations Took: 3
885000 Check Took: 2
886000 Iterations Took: 2
886000 Check Took: 2
887000 Iterations Took: 2
887000 Check Took: 1
888000 Iterations Took: 2
888000 Check Took: 2
889000 Iterations Took: 3
889000 Check Took: 2
890000 Iterations Took: 2
890000 Check Took: 2
891000 Iterations Took: 2
891000 Check Took: 2
892000 Iterations Took: 2
892000 Check Took: 2
893000 Iterations Took: 3
893000 Check Took: 2
894000 Iterations Took: 2
894000 Check Took: 2
895000 Iterations Took: 2
895000 Check Took: 2
896000 Iterations Took: 2
896000 Check Took: 2
897000 Iterations Took: 2
897000 Check Took: 2
898000 Iterations Took: 3
898000 Check Took: 2
899000 Iterat

In [12]:
# average across repeated cycled
for team in records:
    score = []
    if pattern_found: 
        for i in range(repeat_point, len(rank_history)):
            score += [score_history[i][team]]
    else:
        score = [x[team] for x in averaged_score_history]
    scores[team] = list(map(mean, zip(*score)))
    scores[team][0] = sum(scores[team][1:]) 
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ('My Goldfish Computer Poll!\n'
          + 'It only takes into account games played this season (as if it has the memory of a goldfish, thus the name), looking only at margin and quality of wins and losses individually:\n\n'
          + '|Rank|Score|Team|AP Rank|Past Rank|Score Change|Breakdown|\n'
          + '|-|-|-|-|-|-|-|\n')
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(0, len(scores[team]) - 1):
        explanation += (str(round(scores[team][j + 1])) + ' from #'
                        + str(new_rank_orders.index(records[team][j][0]) + 1) + ' '
                        + str(records[team][j][0]) 
                        + '(' + str(records[team][j][1]) + '), ')
    #explanation = 'See Link  '
        
    score_change = round(scores[team][0]) - int(last_week_table[team]['Score'])
    if score_change > 0:
        score_change_text = '+' + str(score_change)
    else:
        score_change_text = str(score_change)
        
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + AP_Ranks[team] +'|'
            + last_week_table[team]['Rank'] + '|'
            + score_change_text + '|'
            + explanation[0:-2] + '\n')
    output += line
print(output)
with open('Output - '
        + str(YEAR) 
          + '-' 
          + str(WEEK) + '.md', 'w') as text_file:
    text_file.write(output)

My Goldfish Computer Poll!
It only takes into account games played this season (as if it has the memory of a goldfish, thus the name), looking only at margin and quality of wins and losses individually:

|Rank|Score|Team|AP Rank|Past Rank|Score Change|Breakdown|
|-|-|-|-|-|-|-|
|1|1899|Coastal Carolina(10-0)|11|6|+748|31 from #122 Kansas(15), 21 from #128 FCS Team(22), 97 from #94 Arkansas State(29), 376 from #6 Louisiana(3), 195 from #46 Georgia Southern(14), 225 from #55 Georgia State(51), 79 from #100 South Alabama(17), 236 from #30 Appalachian State(11), 50 from #117 Texas State(35), 589 from #4 BYU(5)
|2|1764|Notre Dame(10-0)|2|1|-50|72 from #102 Duke(14), 45 from #121 South Florida(52), 91 from #93 Florida State(16), 73 from #89 Louisville(5), 297 from #33 Pittsburgh(42), 99 from #91 Georgia Tech(18), 541 from #3 Clemson(1.0), 190 from #48 Boston College(14), 298 from #21 North Carolina(14), 59 from #112 Syracuse(24)
|3|1746|Clemson(9-1)|4|2|+222|299 from #27 Wake Forest(24), 24 

In [11]:
ranks = []
for i in range (5):
    ranks.append(sorted(scores, key=score_history[-i].__getitem__, reverse=True))
for i in range(len(ranks[0])):
    row = str(i) + ' '
    for j in range (5):
        row += ranks[j][i] + ' '
    print(row)

0 Clemson Coastal Carolina Coastal Carolina Coastal Carolina Coastal Carolina 
1 Coastal Carolina Notre Dame Clemson Clemson Notre Dame 
2 Notre Dame Clemson Notre Dame Notre Dame Clemson 
3 BYU BYU BYU BYU BYU 
4 Cincinnati Cincinnati Cincinnati Cincinnati Cincinnati 
5 Louisiana Louisiana Louisiana Louisiana Louisiana 
6 Miami Miami Miami Miami Miami 
7 Iowa State Iowa State Iowa State Iowa State Iowa State 
8 Alabama Alabama Alabama Alabama Alabama 
9 Ohio State Ohio State Ohio State Ohio State Ohio State 
10 Oklahoma Oklahoma Oklahoma Oklahoma Oklahoma 
11 San José State San José State San José State San José State San José State 
12 Indiana Indiana Indiana Indiana Indiana 
13 Tulsa Tulsa NC State Tulsa Buffalo 
14 Boise State Oklahoma State Oklahoma State Boise State Boise State 
15 Buffalo Boise State Boise State Buffalo Oklahoma State 
16 Oklahoma State Buffalo UCF UCF Tulsa 
17 Colorado Colorado Buffalo Oklahoma State Florida 
18 Texas Texas Tulsa Colorado Colorado 
19 Florida 